### Spectrogram based Transfer Learning

#### Data cleanup

In [1]:
import matplotlib.pyplot as plt
import os
import subprocess

def run_preprocess(root, length, split):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            subprocess.call(["./preprocess", os.path.join(subdir, directory), length, split])
        break

In [2]:
def load_features(root, split):
    spect_data = []
    spect_label = []
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            file_path = os.path.join(subdir, directory, "split", split, "spect")
            for filename in os.listdir(file_path):
                x = plt.imread(os.path.join(file_path, filename))
                spect_data.append(x)
                spect_label.append(directory)
        break
    return spect_data, spect_label

In [3]:
import shutil

def cleanup_split(root):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            shutil.rmtree(os.path.join(subdir, directory, "split"), ignore_errors = True)
        break

In [4]:
cleanup_split('audio-train-transfer')
run_preprocess('audio-train-transfer', "15", "3")

#### Build the model

In [5]:
import keras
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(540, 960, 3)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(10))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(118))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Using TensorFlow backend.


#### Load pretrained model weights

In [6]:
model.load_weights('NN_Weights_210_3/spect_model_weights_4.h5')

In [7]:
from keras.models import Model

transfer_model = Model(inputs = model.input, outputs=model.get_layer('flatten_1').output)

#### Load spectrograms as matrices

In [8]:
X, y = load_features('audio-train-transfer', "3")

In [9]:
import numpy as np

X_SVM = []
for sample in range(len(X)):
    x_exp = np.expand_dims(X[sample], axis = 0)
    transfer_features = transfer_model.predict(x_exp)
    X_SVM.append(transfer_features)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_SVM, y, test_size=0.25, random_state=42)

#### One-hot encode the inputs

In [11]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

l_enc = LabelEncoder()
l_enc.fit(y_train)
y_train_enc = l_enc.transform(y_train)
y_train_norm = np_utils.to_categorical(y_train_enc)

l_enc.fit(y_test)
y_test_enc = l_enc.transform(y_test)
y_test_norm = np_utils.to_categorical(y_test_enc)

#### Remove unnecessary variables from memory (To clear memory)

In [12]:
# del X, y, X_SVM, y_train_enc, y_test_enc, y_train, y_test

In [34]:
from sklearn import svm

model_SVM = svm.SVC(kernel='rbf', class_weight='balanced', C = 100, gamma = 1e-06)
# model_SVM = svm.SVC(kernel='linear', class_weight='balanced')
X_train_SVM = np.array(X_train).reshape(len(X_train), -1)

In [35]:
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import GridSearchCV

# C_range = np.logspace(-2, 10, 13)
# gamma_range = np.logspace(-9, 3, 13)
# param_grid = dict(gamma=gamma_range, C=C_range)
# cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# grid = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=cv)
# grid.fit(X_train_SVM, y_train_norm)

In [36]:
# print("The best parameters are %s with a score of %0.2f"
#       % (grid.best_params_, grid.best_score_))

In [37]:
# y_train_norm = [np.where(r == 1)[0][0] for r in y_train_norm]

In [38]:
model_SVM.fit(X_train_SVM, y_train_norm)

SVC(C=90, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1e-06, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
from sklearn.metrics import accuracy_score

# y_test_norm = [np.where(r == 1)[0][0] for r in y_test_norm]
X_test = np.array(X_test).reshape(len(X_test), -1)
accuracy_score(y_test_norm, model_SVM.predict(X_test))
# model_SVM.predict(X_test)

0.125